# Tutorial 7: Compare Feature Extraction vs. Fine-Tuning

In [ ]:
# Step 1: Importing required libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [ ]:
# Step 2: Load and preprocess the CIFAR-10 dataset
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Step 3: Build and train a model using VGG16 for feature extraction
# Load Pretrained Model (VGG16) and freeze layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))
for layer in base_model.layers:
    layer.trainable = False

# Add New Classification Layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

# Create and Compile Model
model_feature_extraction = Model(inputs=base_model.input, outputs=output)
model_feature_extraction.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
history_feature_extraction = model_feature_extraction.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=32)

In [ ]:
# Step 4 : Load Pretrained Model (ResNet50) and freeze all layers initially for fine tuning
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32,32,3))
for layer in base_model.layers:
    layer.trainable = False

# Add New Classification Layers
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

# Create Model and Compile (initial compile)
model_finetune = Model(inputs=base_model.input, outputs=output)
model_finetune.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-Tune the Last Few Layers: Unfreeze last 5 layers
for layer in base_model.layers[-5:]:
    layer.trainable = True

# Recompile with a potentially lower learning rate (1e-5 in the tutorial)
model_finetune.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
history_finetune = model_finetune.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=32)

In [ ]:
# Step 5 : Evaluate both models
loss_feature_extraction, acc_feature_extraction = model_feature_extraction.evaluate(x_test, y_test)
loss_finetune, acc_finetune = model_finetune.evaluate(x_test, y_test)

# Compare Results
print(f"Feature Extraction Accuracy: {acc_feature_extraction * 100:.2f}%")
print(f"Fine-Tuning Accuracy: {acc_finetune * 100:.2f}%")